| パラメーター | 意味 | 補足   |
|------|------|-------|--------|
|   q  | 銘柄コード|           |
|   x  | 市場|    TYOで問題ない       |
|   i  | ティック|　秒           |
|   p  | 期間|　1Y, 2M, 3d, 4m    |
|   f  | ほしい値|　d=日時,c=終値,o=始値,h=高値,l=安値,v=出来高    |

* MARKET_OPEN_MINUTE: 
 取引開始時間(分) 例えば2015年3月27日(金)のデータを取得した場合、起点は0時0分0秒なのでそこから540分ズラした時間が取引開始時間となり、午前9時となる。
* MARKET_CLOSE_MINUTE: 
 取引終了時間(分) OPENと同様。日経平均の場合は15時となる。
* INTERVAL: 
 観測期間(秒)。この場合は300秒(=5分)間隔のティックデータとなる
* COLUMNS: 
 何の情報がどの順番で返されるか。この順番はURLパラメータの順番で指定できない模様。
* DATA_SESSIONS: 
 前場後場が存在する場合、それぞれの寄り引けの時間が格納される。S&Pなど昼休みが存在しない
 場合は空白となる。
 東証の場合は９：００～１１：３０と１２：３０～１５：００
* DATA: ??
* TIMEZONE_OFFSET: 
 時差(分)。基本的にGoogle financeはGMT基準で時間を表示しているのでローカル時間に変更する場合はこの値を足す。となっているが最初からTimeZoneを考慮した値が返ってきていると思う。
* ティックデータの時間情報の計算方法


![google_finance_res-768x339.png](attachment:google_finance_res-768x339.png)

In [28]:
import requests
import pandas as pd
from io import StringIO
pd.set_option('display.max_columns', 225)
pd.set_option('display.max_rows', 100)



def get_data_frame( date ):
    res = requests.get("http://stockexchangearchiver.appspot.com/archive/csv/" + date)
    csv = StringIO( res.text.replace("<br>", "\n") )
    df = pd.read_csv( csv )
    return df

def get_diff( df ):
    return
    
def get_chain( df ):
    return

if __name__ == "__main__":
    date = "2018.04.18"
    df = get_data_frame( date )
    
    print( df.columns[1] )
    s = df[ df.columns[1] ]
    l = []
    for i in len( s ):
        if i == 0:
            l.append( 0 )
        else
            s[ i ] = s[ i ] - s[ i-1 ]
    print( df[df.columns[1]] )
    
    #display( df )




協和キリン -- 4151
0     2389.0
1     2400.0
2     2403.0
3     2392.0
4     2381.0
5     2366.0
6     2365.0
7     2356.0
8     2355.0
9     2351.0
10    2349.0
11    2347.0
12    2352.0
13    2360.0
14    2359.0
15    2361.0
16    2348.0
17    2350.0
18    2345.0
19    2348.0
20    2347.0
21    2347.0
22    2350.0
23    2351.0
24    2354.0
25    2353.0
26    2351.0
27    2355.0
28    2349.0
29    2346.0
30    2339.0
31    2343.0
32    2344.0
33    2344.0
34    2342.0
35    2350.0
36    2352.0
37    2351.0
38    2351.0
39    2352.0
40    2357.0
41    2356.0
42    2359.0
43    2359.0
44    2360.0
45    2364.0
46    2360.0
47    2361.0
48    2360.0
49    2361.0
50    2365.0
51    2366.0
52    2365.0
53    2367.0
54    2365.0
55    2369.0
56    2370.0
57    2368.0
58    2366.0
59      -1.0
Name: 協和キリン -- 4151, dtype: float64


In [6]:
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

class Stock:
    url = "https://www.google.com/finance/getprices"
    interval = 60 * 5 #sec
    market = "TYO"
    period = "1d"
    column = "d,c"
    market_duration_minute = 360 #休憩を含む
    def __init__(self, code, label, name):
        self.code = code
        self.label = label
        self.name = name
        self.prices = []
        self.timestamps = []
        
    

    def fetch_prices(self):
        params = {
            'q': self.code,
            'i': Stock.interval,
            'x': Stock.market,
            'p': Stock.period,
            'f': Stock.column
        }

        #http リクエスト
        r = requests.get(Stock.url, params=params)

        #print( r.text )
        
        #http レスポンスを行単位に切る
        lines = r.text.splitlines()
        columns = lines[4].split("=")[1].split(",")

        #ここからが株価
        prices = lines[8:]

        #レスポンスの１日目のタイムスタンプをdatetimeに
        base_time = 0
        self.prices = []
        self.timestamps = []
        for i in range( len(prices) ):
            cols = prices[ i ].split(",")
            if 'a' in cols[0]:
                if base_time != 0: break
                base_time = int( cols[0].lstrip('a') )
                d = datetime.fromtimestamp( base_time )
            else:
                ofst = int( cols[0] )
                if not self.within_a_day( ofst ):
                    break
                d = datetime.fromtimestamp( base_time + ofst * Stock.interval )
    
            if  self.just_begining_time(d) or self.just_end_time(d):
                #開始時間ピッタリのやつはおかしい。
                #終了時間ピッタリのやつはないのもあるので除外。
                pass
            else:
                self.prices.append( float(cols[1]) )
                self.timestamps.append(d.strftime('%Y-%m-%d %H:%M:%S'))
            
        return self.prices, self.timestamps
    
    def plot_xy( self ):
        x = np.arange( len(self.prices) )
        y = np.array( self.prices )
        return x, y
    
    def within_a_day( self, ofst ):
        return (ofst * Stock.interval) < (Stock.market_duration_minute * 60)
        
    def just_begining_time( self, d ):
        return (d.hour == 9 and d.minute == 0) or (d.hour == 12 and d.minute == 30 )
    
    def just_end_time( self, d ):
        return (d.hour == 15 and d.minute == 0 )
                

def get_stock( no ):
    s = Stock( no, "", "" )
    s.fetch_prices()
    return s

def print_stock():
    s0 = get_stock( 6758 )
    s1 = get_stock( 6762 )
    s2 = get_stock( 3105 )
    print( "{}:{} {}:{} {}:{}".format( len(s0.prices), len(s0.timestamps), len(s1.prices), len(s1.timestamps), len(s2.prices), len(s2.timestamps)))
    for i in range( len( s1.prices )):
        
        print( "-----" + str(i) + "----" )
        print( "{}  |  {}  |  {}".format( s0.timestamps[i], s1.timestamps[i], s2.timestamps[i]))
        print( "{}  |  {}  |  {}".format( s0.prices[i], s1.prices[i], s2.prices[i]))

if __name__ == '__main__':
    print_stock()
    pass
    

59:59 59:59 57:57
-----0----
2018-04-17 09:05:00  |  2018-04-17 09:05:00  |  2018-04-17 09:05:00
5337.0  |  9680.0  |  1456.0
-----1----
2018-04-17 09:10:00  |  2018-04-17 09:10:00  |  2018-04-17 09:10:00
5352.0  |  9700.0  |  1460.0
-----2----
2018-04-17 09:15:00  |  2018-04-17 09:15:00  |  2018-04-17 09:15:00
5337.0  |  9680.0  |  1462.0
-----3----
2018-04-17 09:20:00  |  2018-04-17 09:20:00  |  2018-04-17 09:20:00
5340.0  |  9640.0  |  1457.0
-----4----
2018-04-17 09:25:00  |  2018-04-17 09:25:00  |  2018-04-17 09:25:00
5347.0  |  9700.0  |  1458.0
-----5----
2018-04-17 09:30:00  |  2018-04-17 09:30:00  |  2018-04-17 09:30:00
5346.0  |  9680.0  |  1455.0
-----6----
2018-04-17 09:35:00  |  2018-04-17 09:35:00  |  2018-04-17 09:35:00
5335.0  |  9690.0  |  1456.0
-----7----
2018-04-17 09:40:00  |  2018-04-17 09:40:00  |  2018-04-17 09:40:00
5337.0  |  9680.0  |  1454.0
-----8----
2018-04-17 09:45:00  |  2018-04-17 09:45:00  |  2018-04-17 09:45:00
5335.0  |  9660.0  |  1454.0
-----9----

IndexError: list index out of range

In [4]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re

class Nikkei225:
    # アクセスするURL
    url = "https://indexes.nikkei.co.jp/nkave/index/component?idx=nk225"
    
    def __init__( self ):
        self.title = ""
        self.date = ""
        self.stocks = []
        self.soup = None
        
    def fetch_date( self ):
        divs = self.soup.find_all("div", class_="last-update" )
        assert len(divs)==1, "no last-update class found."
        self.date = divs[0].string
        
    def fetch_stock( self ):
        self.divs = self.soup.find_all("div", class_=re.compile("row component-list.*") )
        
        for tag in self.divs:
            # classの設定がされていない要素は、tag.get("class").pop(0)を行うことのできないでエラーとなるため、tryでエラーを回避する
            try:
                code_ = tag.find_all("div", class_="col-xs-3 col-sm-1_5")[0].string
                label_ = tag.find_all("div", class_="col-xs-9 col-sm-2_5")[0].string
                company_ = tag.find_all("div", class_="hidden-xs col-sm-8")[0].string
                self.stocks.append( [code_, label_, company_] )
            except:
                # パス→何も処理を行わない
                pass
        
        
    def fetch( self ):
        # URLにアクセスする htmlが帰ってくる → <html><head><title>経済、株価、ビジネス、政治のニュース:日経電子版</title></head><body....
        html = urllib.request.urlopen(self.url)
        
        # htmlをBeautifulSoupで扱う
        self.soup = BeautifulSoup(html, "html.parser")

        # タイトル要素を取得する → <title>経済、株価、ビジネス、政治のニュース:日経電子版</title>
        title_tag = self.soup.title

        # 要素の文字列を取得する → 経済、株価、ビジネス、政治のニュース:日経電子版
        self.title = title_tag.string
              
        self.fetch_date()
        self.fetch_stock()

def print_stocks():
    n = Nikkei225()
    n.fetch()
    print( n.title )
    print( n.date )
    for i, s in enumerate( n.stocks ):
        print( "{:3}: {}".format( i, s ) )
        
if __name__ == '__main__':
    print_stocks()
    pass
    
                             
                             

銘柄一覧 － 日経平均プロフィル
更新日付：2018.04.17
  0: ['4151', '協和キリン', '協和発酵キリン（株）']
  1: ['4502', '武田', '武田薬品工業（株）']
  2: ['4503', 'アステラス', 'アステラス製薬（株）']
  3: ['4506', '大日本住友', '大日本住友製薬（株）']
  4: ['4507', '塩野義', '塩野義製薬（株）']
  5: ['4519', '中外薬', '中外製薬（株）']
  6: ['4523', 'エーザイ', 'エーザイ（株）']
  7: ['4568', '第一三共', '第一三共（株）']
  8: ['4578', '大塚ＨＤ', '大塚ホールディングス（株）']
  9: ['3105', '日清紡ＨＤ', '日清紡ホールディングス（株）']
 10: ['6479', 'ミネベア', 'ミネベアミツミ（株）']
 11: ['6501', '日立', '（株）日立製作所']
 12: ['6503', '三菱電', '三菱電機（株）']
 13: ['6504', '富士電機', '富士電機（株）']
 14: ['6506', '安川電', '（株）安川電機']
 15: ['6674', 'ＧＳユアサ', '（株）ジーエス・ユアサ\u3000コーポレーション']
 16: ['6701', 'ＮＥＣ', '日本電気（株）']
 17: ['6702', '富士通', '富士通（株）']
 18: ['6703', 'ＯＫＩ', '沖電気工業（株）']
 19: ['6724', 'エプソン', 'セイコーエプソン（株）']
 20: ['6752', 'パナソニック', 'パナソニック（株）']
 21: ['6758', 'ソニー', 'ソニー（株）']
 22: ['6762', 'ＴＤＫ', 'ＴＤＫ（株）']
 23: ['6770', 'アルプス', 'アルプス電気（株）']
 24: ['6773', 'パイオニア', 'パイオニア（株）']
 25: ['6841', '横河電', '横河電機（株）']
 26: ['6857', 'アドテスト', '（株）アドバンテスト']
 27: ['6902', 'デンソー', '（

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
from ipywidgets import *

n = Nikkei225()
n.fetch()

lst = []

#ラベル
lbls = []
ddd = range( 10 )
menu = None

for i, c in enumerate( n.stocks ):
    lbls.append( "- " + c[1] + " (" + c[0] + ")" )

def minmax( l, margin = 4 ):
    min = l.min()
    max = l.max()
    return min-margin, max+margin


def plot_prices():
    y_min = sys.maxsize
    y_max = -sys.maxsize
    xs = []
    ys = []
    for s in lst:
        x,y = s.plot_xy()   
        xs.append( x )
        ys.append( y )
        min, max = minmax( y )
        if min < y_min: 
            y_min = min
        if max > y_max:
            y_max = max
    
    plt.ylim( y_min, y_max )
    for i in range( len(lst) ):
        plt.plot(xs[i],ys[i])
    plt.show()
    #print( y )

def change_label( i, src, dst ):
    cp = list( lbls )
    cp[ i ] = cp[ i ].replace(src, dst)
    print( cp[ i ] )
    return cp

def is_shown( i ):
    return "+" in lbls[i]

def on_value_change( X ):
    idx = lbls.index( X )
    d = n.stocks[ idx ]
    if is_shown( idx ):
        ddd = change_label( idx, "+", "-")
        pass
    else:
        #s = Stock( d[0], d[1], d[2] )
        #s.fetch_prices()
        #lst.append( s )
        ddd = change_label( idx, "-", "+")
        #plot_prices()
        pass
    del_menu()
    lbls = ddd
    create_menu()

    
def create_menu():
    menu = Dropdown(options=lbls)
    display( interactive(on_value_change, X=menu) )

def del_menu():
    menu.close()
    del menu

geo={'USA':['CHI','NYC'],'Russia':['MOW','LED']}

def print_city(city):
    print ( city )

def select_city(country):
    cityW.options = geo[country]

def test():
    scW = widgets.Dropdown(options=geo.keys())
    init = scW.value
    cityW = widgets.Dropdown(options=geo[init])
    j = widgets.interactive(print_city, city=cityW)
    i = widgets.interactive(select_city, country=scW)
    display(i)
    display(j)

        
if __name__ == '__main__':
    create_menu()
    

NameError: name 'Nikkei225' is not defined

In [12]:
#from urllib2 import *
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import pandas as pd


# アクセスするURL
url = "https://indexes.nikkei.co.jp/nkave/index/component?idx=nk225"

# URLにアクセスする htmlが帰ってくる → <html><head><title>経済、株価、ビジネス、政治のニュース:日経電子版</title></head><body....
html = urllib.request.urlopen(url)

# htmlをBeautifulSoupで扱う
soup = BeautifulSoup(html, "html.parser")

# タイトル要素を取得する → <title>経済、株価、ビジネス、政治のニュース:日経電子版</title>
title_tag = soup.title

# 要素の文字列を取得する → 経済、株価、ビジネス、政治のニュース:日経電子版
title = title_tag.string

# タイトル要素を出力
#print( title_tag )

# タイトルを文字列を出力
print( title )

divs = soup.find_all("div")

#help( divs[0] )

date = ""
stocks = []

for tag in divs:
    # classの設定がされていない要素は、tag.get("class").pop(0)を行うことのできないでエラーとなるため、tryでエラーを回避する
    try:
        # tagの中からclass="n"のnの文字列を摘出します。複数classが設定されている場合があるので
        # get関数では配列で帰ってくる。そのため配列の関数pop(0)により、配列の一番最初を摘出する
        # <span class="hoge" class="foo">  →   ["hoge","foo"]  →   hoge
        strings_ = tag.get("class")
        string_ = ""
        for s in strings_:
            string_ += s
            string_ += ' '#末尾に空白が入る。
            
        if string_ in "row component-list " or string_ in "row component-list list-even ":

            code_ = ""
            label_ = ""
            company_ = ""
            tags = tag.find_all("div")
            for child in tags:
                string_ = child.get("class").pop(0)
                
                #print( type( child.string ) )
                if string_ in "col-xs-3 col-sm-1_5": code_ = "{0}".format(child.string)
                if string_ in "col-xs-9 col-sm-2_5": label_ = "{0}".format(child.string)
                if string_ in "hidden-xs col-sm-8": company_ = "{0}".format(child.string)

            
            if len( code_ ) != 0:
                s = Stock( code_, label_, company_ )
                s.fetch_prices()
                #print( s.prices )
                l = [code_,label_]
                l = l + s.prices
                stocks.append( l )
                #stocks.append( [code_,label,company_] )
                #print( code_ + label_ + company_)
                #break

        elif string_ in "last-update ":
            date = tag.string
    except:
        # パス→何も処理を行わない
        pass
    
print( date )

#stocks

df = pd.DataFrame( stocks )
df.to_csv( "日経２２５ " + date + ".csv", index=False)
df


銘柄一覧 － 日経平均プロフィル
更新日付：2018.04.12


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,4151,協和キリン,2355.0,2359.0,2361.0,2361.0,2350.0,2348.0,2344.0,2343.0,...,2331.0,2336.0,2339.0,2336.0,2337.0,2340.0,2340.0,2341.0,2343.0,2342.0
1,4502,武田,5010.0,5028.0,5036.0,5034.0,5028.0,5030.0,5037.0,5043.0,...,5068.0,5068.0,5068.0,5061.0,5067.0,5068.0,5068.0,5068.0,5071.0,5075.0
2,4503,アステラス,1571.5,1583.0,1592.5,1593.0,1590.5,1589.0,1589.5,1589.0,...,1577.5,1578.0,1577.0,1577.5,1579.5,1582.5,1583.0,1583.5,1584.0,1584.0
3,4506,大日本住友,1957.0,1957.0,1961.0,1960.0,1954.0,1954.0,1957.0,1953.0,...,1935.0,1937.0,1937.0,1937.0,1937.0,1939.0,1939.0,1939.0,1939.0,1937.0
4,4507,塩野義,5471.0,5480.0,5522.0,5529.0,5517.0,5514.0,5505.0,5505.0,...,5506.0,5507.0,5507.0,5504.0,5504.0,5505.0,5503.0,5506.0,5518.0,5512.0
5,4519,中外薬,5620.0,5640.0,5680.0,5690.0,5670.0,5680.0,5660.0,5660.0,...,5570.0,5560.0,5560.0,5560.0,5570.0,5580.0,5580.0,5580.0,5610.0,5610.0
6,4523,エーザイ,6699.0,6666.0,6766.0,6784.0,6769.0,6748.0,6753.0,6736.0,...,6674.0,6687.0,6690.0,6673.0,6683.0,6684.0,6685.0,6697.0,6702.0,6699.0
7,4568,第一三共,3699.0,3700.0,3720.0,3718.0,3695.0,3696.0,3701.0,3704.0,...,3665.0,3665.0,3666.0,3661.0,3657.0,3664.0,3664.0,3672.0,3675.0,3676.0
8,4578,大塚ＨＤ,5313.0,5332.0,5373.0,5369.0,5355.0,5349.0,5347.0,5342.0,...,5316.0,5332.0,5332.0,5334.0,5335.0,5341.0,5343.0,5344.0,5349.0,5346.0
9,3105,日清紡ＨＤ,1448.0,1454.0,1443.0,1439.0,1440.0,1445.0,1450.0,1449.0,...,1443.0,1444.0,1443.0,1442.0,1442.0,1443.0,1442.0,1442.0,1442.0,1442.0


In [13]:
s = ""
s = "{0}".format( "test")
print( s )

test
